In [75]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from itertools import zip_longest
import time,random , os
import pandas as pd
import urllib.parse

In [76]:
def make_driver(headless=False):
    from selenium.webdriver.chrome.options import Options
    options = Options()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1280,900")
    #options.add_argument("--disable-images") 
    # options.add_argument("--blink-settings=imagesEnabled=false")
    #options.add_argument("--lang=en-US")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)
    driver = webdriver.Chrome(options=options)
    driver.set_page_load_timeout(60)
    driver.implicitly_wait(2)  # petit implicite pour réduire le code
    return driver

In [139]:
df=pd.read_csv("data/all_places.csv")

In [78]:
def scroll_reviews(driver, scrolls=25, pause=1.3):
    """Scroll inside the reviews panel to load more reviews."""
    panel = driver.find_element(By.XPATH, "//div[contains(@class,'m6QErb DxyBCb kA9KIf dS8AEf XiKgde')]")
    last_height = 0
    for _ in range(scrolls):
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", panel)
        time.sleep(pause)
        new_height = driver.execute_script("return arguments[0].scrollHeight", panel)
        if new_height == last_height:
            break
        last_height = new_height

In [79]:
def extract_rating(card):
    """Retourne la note trouvée, sinon '' """
    try:
        elem = card.find_element(By.XPATH, ".//span[contains(@class,'kvMYJc') or contains(@class,'fontBodyLarge fzvQIb')]")
        rating = elem.get_attribute("aria-label") or elem.text
        return rating.strip()
    except:
        return ""

In [ ]:
def get_reviews(driver, place_url, max_reviews=50):
    reviews = []
    
    driver.get(place_url)
    wait = WebDriverWait(driver, 10)

    try:
        # Click on the "Reviews" button
        reviews_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@aria-label,'Avis') or contains(@aria-label,'Reviews')]")))
        reviews_btn.click()

        # Scroll to load reviews
        scroll_reviews(driver)

        # Collect review cards
        cards = driver.find_elements(By.XPATH, "//div[contains(@class,'jftiEf')]")
        if len(cards)>50:
            max_reviews=50
        else:
            max_reviews=len(cards)
        for card in cards[:max_reviews]:
            try:
                name = card.find_element(By.XPATH, ".//div[@class='d4r55 fontTitleMedium']").text
            except:
                name = ""
            rating = extract_rating(card)
            try:
                date = card.find_element(By.XPATH, ".//span[contains(@class,'xRkPPb') or contains(@class, 'rsqaWe')]").text
            except:
                date = ""

            try:
                text = card.find_element(By.XPATH, ".//span[contains(@class,'wiI7pd')]").text
            except:
                text = ""

            reviews.append({"name": name, "note": rating, "date": date,"date modifié": str(date).split("sur")[0] ,"review text": text})
            
    except Exception as e:
        print(f"[WARN] Could not scrape {place_url} – {e}")
    finally:
        driver.quit()
    return reviews


In [ ]:
# url=df.iloc[0]["link"]
# category=df.iloc[0]["category"]
# city=df.iloc[0]["city"]

In [145]:
reviews=[]
for i in range(len(data)):
    url=data.iloc[i]["link"]
    print(f"scraping {data.iloc[i]['category']} of {data.iloc[i]['place name']} from {data.iloc[i]['city']}")
    driver=make_driver()
    results=get_reviews(driver, url)
    results=pd.DataFrame(results)
    for j in range(len(results)):
        reviews.append({
            "city":data.iloc[i]["city"],
            "category":data.iloc[i]["category"],
            "place name":data.iloc[i]["place name"],
            "name": results.iloc[j]["name"],
            "date" : results.iloc[j]["date modifié"],
            "review text":results.iloc[j]["review text"]
        })


scraping Restaurant of Marine Club from Saidia
scraping Restaurant of Twareg : Restaurant & Lounge from Saidia
scraping Restaurant of Hamouda from Saidia
scraping Restaurant of O'Ptit Belge from Saidia
scraping Restaurant of Mardouma : Tanjia, Viandes grillées à la façon Mardouma from Saidia
scraping Restaurant of Café Hambourg from Saidia
scraping Restaurant of Snack El Fakher pizza 🍕 from Saidia
scraping Restaurant of Snack El Bahja from Saidia
scraping Restaurant of Étoile d'Or : Café Restaurant Pizzeria from Saidia
scraping Restaurant of Le Maris Pool Bar from Saidia
scraping Restaurant of Le Refuge from Saidia
scraping Restaurant of GÜRME - Berliner Kebab from Saidia
scraping Restaurant of Marina de Saïdia from Saidia
scraping Restaurant of Primo Beach from Saidia
scraping Restaurant of La Taberna del Puerto from Saidia
scraping Tourist Attraction of Saidia Corniche from Saidia
scraping Tourist Attraction of Aquaparc Alpamare Saïdia from Saidia
scraping Tourist Attraction of Plage